查看cuda版本 cuda>=10.0

In [ ]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


安装mxnet和gluonnlp

In [ ]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 27kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install --upgrade gluonnlp

     |████████████████████████████████| 256kB 7.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470046 sha256=384e1d323d35f39accbf79889c7c664d0b07b7570011814303ec2c40cc127e7f
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


查看硬件信息gpu

In [ ]:
!nvidia-smi

Sun Jul 12 23:46:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

挂载google云端硬盘

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


根据自己路径进行修改

In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [ ]:
cd NLP/

/content/gdrive/My Drive/Colab Notebooks/NLP


In [ ]:
#导入相关库
import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
from bert import data
import random

加载模型

In [ ]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)
filename="./base_2.params"
nlp.utils.load_parameters(bert_classifier,filename,ctx=ctx)

load a sentence to infer

In [ ]:

field_separator = nlp.data.Splitter('\t')
data_inference_row = nlp.data.TSVDataset(filename='test.tsv',
                field_separator=field_separator)

bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
max_len = 128
all_labels = ["0", "1"]
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_inference=data_inference_row.transform(transform)
batch_size = 1
data_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_inference],
                      batch_size=batch_size,
                      shuffle=True)


In [ ]:
data_loader = mx.gluon.data.DataLoader(data_inference, batch_sampler=data_sampler)
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(data_loader):
  token_ids = token_ids.as_in_context(ctx)
  valid_length = valid_length.as_in_context(ctx)
  segment_ids = segment_ids.as_in_context(ctx)
  label = label.as_in_context(ctx)
  out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
  print(data_inference_row[0][0])
  print('label: ',data_inference_row[0][1])
  if (out.argmax(axis=1))[0]==1:
    print('inference: 1')
  else:
    print('inference: ')

Another fantastic offering from the Monkey Island team and though it was a long time coming and had to survive the departure of Ron Gilbert it's another worthy installment. My only gripe is that it was a little short seeming in comparison to the previous two, though that might be because of a glorious lack of disk-swapping. Roll on MI4.
label:  1
inference: 1
